# Detect DNS Exfiltration using Deep Learning

This notebook uses a pre-trained model to detect if a DNS request is a DNS exfiltration request. The pre-trained model is a deep learning model that takes in current DNS request along with previous 'x' number of requests between the same src and subdomain. The deep learning layers consume text input along with additional features such as entropy, length for the current request and aggregated features such as entropy_avg and size_avg in past few requests. 

The model outputs a probability score and the threshold for determining whether it is DNS exfiltration request or not is set at 0.5. Higher the `pred_is_dns_data_exfiltration_proba` probability it is more likely the DNS request is exfiltration.  The notebook has apply() method implemented and fit() method unimplemented since it uses pre-trained model for making predictions. 

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [2]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader,TensorDataset,Dataset
from torch.autograd import Variable as V
import time
from sklearn.metrics import classification_report, confusion_matrix,roc_curve, auc
from sklearn.model_selection import train_test_split
import string
import torch.nn as nn
import collections
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.pipeline import Pipeline
from collections import Counter
import torch.optim as optim
import math
MODEL_DIRECTORY = "/srv/app/model/data/detect_dns_data_exfiltration_using_pretrained_model_in_dsdl/"

In [3]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.22.1
pandas version: 1.5.1


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer mode=stage algo=barebone epochs=10 batch_size=1 s from feature_* into app:barebone_model

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [4]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [5]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("dns_exfil5")
print(df.describe())
print(param)

       is_exfiltration       rank
count        10.000000  10.000000
mean          4.800000   4.800000
std           3.084009   3.084009
min           1.000000   1.000000
25%           2.000000   2.000000
50%           4.500000   4.500000
75%           7.750000   7.750000
max           9.000000   9.000000
{'options': {'params': {'mode': 'stage', 'algo': 'dns_exfil5'}, 'args': ['is_exfiltration', 'src', 'query', 'rank'], 'target_variable': ['is_exfiltration'], 'feature_variables': ['src', 'query', 'rank'], 'model_name': 'dns_exfil5', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'handle_new_cat': 'default', 'max_distinct_cat_values': '100', 'max_distinct_cat_values_for_classifiers': '100', 'max_distinct_cat_values_for_scoring': '100', 'max_fit_time': '600', 'max_inputs': '100000', 'max_memory_usage_mb': '4000', 'max_model_size_mb': '30', 'max_score_time': '600', 'use_sampling': 'true'}, 'kfold_cv': None}, 'feature_variables': ['src', 'query', 'rank'], 'target_variables': ['is_exfiltrati

## Stage 2 - create and initialize a model

In [6]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
device='cpu'
class DNSExfiltration(nn.Module):
    def __init__(self,input_size):
        super().__init__()
        self.layer_1 = nn.Linear(input_size, 128) 
        self.layer_2 = nn.Linear(128, 128)
        self.layer_out = nn.Linear(128, 1) 
        
        
        self.relu = nn.ReLU()
        self.sigmoid =  nn.Sigmoid()
        self.dropout = nn.Dropout(p=0.5)
        

    def forward(self, input):
        x = self.relu(self.layer_1(input))
        x = self.dropout(x)
        x = self.relu(self.layer_2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.layer_out(x))
        return x
    

keys = list(string.printable.strip())
print (len(list(string.printable.strip())))
    
def init(df,param):
    model = DNSExfiltration(98)
    model.load_state_dict(torch.load(MODEL_DIRECTORY+'detect_dns_data_exfiltration_using_pretrained_model_in_dsdl.pt',map_location=torch.device('cpu')))
    model = model.to('cpu')
    model.eval()
    return model

94


In [7]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(init(df,param))

DNSExfiltration(
  (layer_1): Linear(in_features=99, out_features=256, bias=True)
  (layer_2): Linear(in_features=256, out_features=256, bias=True)
  (layer_out): Linear(in_features=256, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (dropout): Dropout(p=0.5, inplace=False)
)


## Stage 3 - fit the model

In [8]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "model trained"}
    return info

In [9]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(None,None,None))

{'message': 'model trained'}


## Stage 4 - apply the model

In [33]:
# apply your model
# returns the calculated results
text_rows = []
size_avg = []
entropy_avg = []

# this method accepts a dataloader and makes predictions in batches
def predict(dataloader,model):
        predict_label_proba = []
        predict_label = []
        for batch in (dataloader):
            
            #convert to 1d tensor
            predictions = model(batch.to('cpu'))
            output  = (predictions >= 0.5).int() 
            predict_label_proba.extend(predictions)
            predict_label.extend(output)
        predict_label = [x.cpu().detach().numpy().item() for x in predict_label]
        predict_label_proba = [x.cpu().detach().numpy().item() for x in predict_label_proba]
        return predict_label_proba,predict_label
    
# this method accepts a DNS request and converts into indexes based on printable characters
def index_chars(x):
    request_chars = {}
    for i in range(len(x)):
        request_chars[keys.index(x[i])] = request_chars.get(keys.index(x[i]), 0) + 1
    text_rows.append(request_chars)

#  calculates entropy of a domain
def entropy(domain):
    p, lns = Counter(domain), float(len(domain))
    return -sum(count / lns * math.log(count / lns, 2) for count in p.values())


# removes the subdomain/domain from the request
def replace_tld(x):
    if x is not None:
        return str(x).rsplit('.', 2)[0]
    else:
        return x
    
# get the subdomain/domain from the request
def get_tld(x):
    without_tld = str(x).rsplit('.', 2)[0]
    return str(x).replace(without_tld,'').lstrip(".")

# compute aggregated features for the same src and subdomain/domain on a window of 10 events
def get_aggregated_features(row,df):
    src = row['src']
    tld = row['tld']
    prev_events = df[(df['src']==src) & (df['tld']==tld)]
    
    size_avg.append(prev_events['len'].mean())
    entropy_avg.append(prev_events['entropy'].mean())

# prepare input df by calculating features
def prepare_input_df(df):
    keys = list(string.printable.strip())

    
    
    df['query'].apply(lambda x: index_chars(x))
    text = pd.DataFrame(text_rows, columns=list(range(0, 94)))
    text.reset_index(drop=True, inplace=True)
    df.reset_index(drop=True, inplace=True)
    text.fillna(0, inplace=True)
    df = pd.concat([text, df], axis=1)


    # request without tld
    df['request_without_domain'] = df['query'].apply(lambda row: replace_tld(row))

    # request without tld
    df['tld'] = df['query'].apply(lambda row: get_tld(row))
    
    # length of domain
    df['len'] = df['request_without_domain'].apply(len)
    
    # entropy
    df['entropy'] = df['request_without_domain'].apply(lambda x: entropy(x))
    
    # take most-recent request
    recent_df = df.loc[df['rank'] == 1]

    # calculate feature by aggregating events

    recent_df.apply(lambda x: get_aggregated_features(x,df),axis=1)
    recent_df['size_avg'] = size_avg
    recent_df['entropy_avg'] = entropy_avg
    return recent_df

    
# apply model on processed dataframe to predict exfiltration
def apply(model,df,param):
    df.drop(['_time'], axis=1,inplace=True, errors='ignore')
    recent_df = prepare_input_df(df)
    input_df = recent_df.drop(['src' ,'query','rank','request_without_domain','tld'], axis=1)
    recent_df.drop(['request_without_domain','tld','len','entropy','size_avg','entropy_avg'], axis=1, inplace=True)
    recent_df.drop(range(0, 94),axis=1,inplace=True)
    input_tensor = torch.FloatTensor(input_df.values)
    dataloader = DataLoader(input_tensor, shuffle=True, batch_size=256)
    predict_is_exfiltration_proba, predict_is_exfiltration = predict(dataloader,model)
    recent_df['pred_is_dns_data_exfiltration_proba'] = predict_is_exfiltration_proba
    recent_df['pred_is_dns_data_exfiltration'] = predict_is_exfiltration
    print(recent_df.columns)
    print(df.columns)
    text_rows.clear()
    size_avg.clear()
    entropy_avg.clear()
    output = pd.merge(recent_df,df,on=['src','query','rank'],how='right')
    return output

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [12]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = DNSExfiltration(98)
    model.load_state_dict(torch.load(MODEL_DIRECTORY+'detect_dns_data_exfiltration_using_pretrained_model_in_dsdl.pt',map_location=torch.device('cpu')))
    model = model.to('cpu')
    model.eval()
    return model

In [13]:
load("")

DNSExfiltration(
  (layer_1): Linear(in_features=99, out_features=256, bias=True)
  (layer_2): Linear(in_features=256, out_features=256, bias=True)
  (layer_out): Linear(in_features=256, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (dropout): Dropout(p=0.5, inplace=False)
)

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code